### Step 1: Load python modules and define input data files

In [1]:
import pygplates
import subduction_convergence as sc
import numpy as np
import os
import matplotlib.pyplot as plt
#from matplotlib import gridspec
#from scipy.io import netcdf_file as netcdf
#import scipy.interpolate as spi
#from mpl_toolkits.basemap import Basemap
#import inpaint
import pandas as pd

%matplotlib inline

#######################################################
# Define Input files for Muller 2016 AREPS model
#######################################################
# Specify the name of the GPlates rotation file
rotation_filename = './MullerData/Global_EarthByte_230-0Ma_GK07_AREPS.rot'

# input topologies to be used with the subduction_convergence script
input_topology_filename = ['./MullerData/Global_EarthByte_230-0Ma_GK07_AREPS_PlateBoundaries.gpmlz',\
                           './MullerData/Global_EarthByte_230-0Ma_GK07_AREPS_Topology_BuildingBlocks.gpmlz']


######################################################
rotation_model = pygplates.RotationModel(rotation_filename)

# specify time range and resolution for plots
threshold_sampling_distance_radians = np.radians(0.5)

# Define the time snapshots at which to get the subduction zone properties
min_time = 0.
max_time = 230.
time_step = 1.

# Set the delta time for velocity calculations
velocity_delta_time = 5.

# Typically the achor plate id should be 0
anchor_plate_id = 0



### Number crunching section

In [2]:
# Do the number crunching
# Create a dataframe that contains all subduction data for full sequence of time steps

# Data frame template defining the column names
DataFrameTemplate = ('lon','lat','conv_rate','conv_obliq','migr_rate',
                     'migr_obliq','arc_length','arc_azimuth',
                     'subducting_plate','overriding_plate','time')

# Create an empty dataframe to concatenate results to
df_AllTimes = pd.DataFrame(columns=DataFrameTemplate)

# list of reconstruction times
reconstruction_times = np.arange(min_time,max_time+1,time_step)

# Iterate over time steps
for reconstruction_time in reconstruction_times:
        
    # Get the subduction kinematics database 
    subduction_data = []
    output_data = sc.subduction_convergence(
                rotation_filename,
                input_topology_filename,
                threshold_sampling_distance_radians,
                reconstruction_time,
                velocity_delta_time,
                anchor_plate_id)

    # Make a flat list of subduction stats to input into the proximity test
    for data in output_data:
        subduction_data.append(data+(reconstruction_time,))
    
    # convert list array to dataframe
    df = pd.DataFrame(subduction_data, columns = DataFrameTemplate)
    
    # append dataframe 
    df_AllTimes = df_AllTimes.append(df)
    

Unable to find the overriding plate of the subducting shared sub-segment "So Melanesian-MG-5-20-07"
    subduction zone feature is missing subduction polarity property or it is set to "Unknown".
Unable to find the overriding plate of the subducting shared sub-segment "So Melanesian-MG-5-20-07"
    subduction zone feature is missing subduction polarity property or it is set to "Unknown".
Unable to find the overriding and subducting plates of the subducting shared sub-segment "Sandwich Trench" at 0.0Ma
    there are not exactly 2 topologies sharing the sub-segment.
Unable to find the overriding and subducting plates of the subducting shared sub-segment "Tonga-Kermadec-MG" at 0.0Ma
    there are not exactly 2 topologies sharing the sub-segment.
Unable to find the overriding and subducting plates of the subducting shared sub-segment "Hikurangi& Kermadec TR" at 0.0Ma
    there are not exactly 2 topologies sharing the sub-segment.
Unable to find the overriding and subducting plates of the su

### Options for saving/loading results to/from file

In [4]:
# Save to csv file
df_AllTimes.to_csv('./results/SubductionTable_%0.0f_%0.0fMa.csv' % (min_time,max_time)) 

